### File Upload Test

In [36]:
import requests
import json
from IPython.display import display, HTML, Markdown, clear_output
from dotenv import load_dotenv
from openai import AzureOpenAI
import os
import base64

load_dotenv(override=True)

import threading

# Function to process and display the streamed response
def process_streamed_response(response):
    buffer = ''
    try:
        for line in response.iter_lines(decode_unicode=True):
            if line:
                print(line)
                if line.startswith('data: '):
                    data_content = line[len('data: '):]
                    # Parse JSON data
                    data_dict = json.loads(data_content)
                    content = data_dict.get('content', '')
                    buffer += content
                    # Clear the output and display the updated content
                    clear_output(wait=True)
                    if buffer.count('```') % 2 == 1:
                        display(Markdown(buffer + '```'))
                    else:
                        display(Markdown(buffer))
        # Ensure the final content is displayed
        # clear_output(wait=True)
        display(Markdown(buffer))
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        response.close()

In [37]:
with open('power_consumption.csv', 'rb') as f:
    data = f.read()
    b64_data = base64.b64encode(data).decode('utf-8')
    body = {
        'file_name': 'power_consumption.csv',
        'file_data': b64_data,
    }
    thread_id = requests.post("http://127.0.0.1:8000/upload_file_and_create_thread", json=body)

In [ ]:
thread_id

In [38]:
message = "Plot out zone 2 power consumption"

In [ ]:
url = 'http://127.0.0.1:8000/run_assistant'

data={
    'thread_id': thread_id.json(), 
    'message': message
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    lthread = threading.Thread(target=process_streamed_response, args=(response,))
    lthread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)